In [1]:
import pandas as pd
import joblib  
import matplotlib.pyplot as plt

# LOAD BOTH files
model = joblib.load('anomaly_model_final.pkl')  
scaler = joblib.load('scaler_final.pkl')        

print("✅ Model & Scaler loaded!")
print(f"Model: {type(model).__name__}")
print(f"Model expects {model.n_features_in_} features")

✅ Model & Scaler loaded!
Model: RandomForestClassifier
Model expects 3 features


In [3]:
def predict_anomaly(temperature, humidity, air_quality):
    """Test model with new values """
    # Create input
    input_data = pd.DataFrame({
        'temperature': [float(temperature)],
        'humidity': [float(humidity)],
        'air_quality': [float(air_quality)]
    })
    
    # SCALE the input!
    input_scaled = scaler.transform(input_data) 
    
    # Get prediction
    is_anomaly = model.predict(input_scaled)[0]      # Use scaled
    confidence = model.predict_proba(input_scaled)[0][1]  # Use scaled
    
    return {
        'is_anomaly': bool(is_anomaly),
        'confidence': float(confidence),
        'anomaly_probability': f"{confidence:.1%}"
    }

In [5]:
print("\n" + "="*60)
print(" MODEL TESTING")
print("="*60)
print("Change values below to test different scenarios:\n")

# TEST DIFFERENT VALUES HERE
test_values = [
    (25.0, 55.0, 70.0, "Perfect normal"),
    (35.0, 64.8, 100.0, "Known anomaly"),
    (30.0, 85.0, 80.0, "High humidity"),
    (40.0, 50.0, 150.0, "Extreme"),
]

for temp, hum, aq, label in test_values:
    result = predict_anomaly(temp, hum, aq)
    status = "🔴 ANOMALY" if result['is_anomaly'] else "✅ NORMAL"
    print(f"{label}:")
    print(f"  {temp}°C, {hum}%, {aq} AQI → {status} ({result['anomaly_probability']})")
    print()


 MODEL TESTING
Change values below to test different scenarios:

Perfect normal:
  25.0°C, 55.0%, 70.0 AQI → ✅ NORMAL (2.5%)

Known anomaly:
  35.0°C, 64.8%, 100.0 AQI → 🔴 ANOMALY (96.5%)

High humidity:
  30.0°C, 85.0%, 80.0 AQI → 🔴 ANOMALY (80.0%)

Extreme:
  40.0°C, 50.0%, 150.0 AQI → 🔴 ANOMALY (100.0%)



In [7]:
print("\n" + "="*60)
print("MANUAL INPUT TESTING")
print("="*60)

while True:
    try:
        print("\nEnter sensor values (or 'q' to quit):")
        temp = input("Temperature (°C): ")
        if temp.lower() == 'q':
            break
        hum = input("Humidity (%): ")
        aq = input("Air Quality (AQI): ")
        
        result = predict_anomaly(float(temp), float(hum), float(aq))
        status = "🔴 ANOMALY" if result['is_anomaly'] else "✅ NORMAL"
        
        print(f"\n📊 Prediction:")
        print(f"   Input: {temp}°C, {hum}%, {aq} AQI")
        print(f"   Result: {status}")
        print(f"   Confidence: {result['anomaly_probability']}")
        
        if result['is_anomaly']:
            print(f"   ⚠️  Alert: Possible sensor issue or environmental problem")
        
    except ValueError:
        print("❌ Please enter valid numbers!")
    except Exception as e:
        print(f"❌ Error: {e}")


MANUAL INPUT TESTING

Enter sensor values (or 'q' to quit):


Temperature (°C):  25
Humidity (%):  70
Air Quality (AQI):  100



📊 Prediction:
   Input: 25°C, 70%, 100 AQI
   Result: 🔴 ANOMALY
   Confidence: 54.5%
   ⚠️  Alert: Possible sensor issue or environmental problem

Enter sensor values (or 'q' to quit):


Temperature (°C):  25
Humidity (%):  50
Air Quality (AQI):  70



📊 Prediction:
   Input: 25°C, 50%, 70 AQI
   Result: ✅ NORMAL
   Confidence: 0.0%

Enter sensor values (or 'q' to quit):


Temperature (°C):  q
